In [1]:
import torch
from tqdm import tqdm
from torchvision import transforms, datasets, models
from torchvision.models import ResNet50_Weights, VGG16_Weights
from torch.utils.data import DataLoader, Subset, random_split
device = torch.device("cuda")
PATH = '../sfddd/imgs/train'

In [2]:
data_transforms = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

dataset = datasets.ImageFolder(PATH, transform=data_transforms)
subset_indices = range(10000)

# Create a Subset instance
subset = Subset(dataset, subset_indices)

# Define the size of the test set
test_size = int(0.3 * len(subset))
train_size = len(subset) - test_size

# Randomly split the dataset into training and test set
train_dataset, test_dataset = random_split(subset, [train_size, test_size])

# Create DataLoaders if needed
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred.data, 1)
    total = y_true.size(0)
    correct = (predicted == y_true).sum().item()
    return 100 * correct / total

In [9]:
# ResNet50
weights = ResNet50_Weights.DEFAULT
model = models.resnet50(weights=weights)
num_classes = 10  # Change this according to your dataset

# Modify final fully connected layer
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, num_classes)
# Define loss function
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 3  # Change this as needed
model = model.to(device)

# check model size
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    acc = 0.0

    for images, labels in tqdm(train_loader):
        # Move inputs and labels to GPU if available
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        try:
            outputs = model(images)
            loss = criterion(outputs, labels)
    
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            acc += calculate_accuracy(outputs, labels)

        except RuntimeError as exception:
          if "out of memory" in str(exception):
              print('WARNING: out of memory, will pass this')     
              torch.cuda.empty_cache()
              continue
          else:
              raise exception
    # Calculate average loss and accuracy over an epoch
    avg_loss = running_loss / len(train_loader)
    avg_acc = acc / len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, ACC: {avg_acc:.4f}")

# test
model.eval()
val_running_loss = 0.0
val_acc = 0.0

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        val_running_loss += loss.item()
        val_acc += calculate_accuracy(outputs, labels)

avg_val_loss = val_running_loss / len(test_loader)
avg_val_acc = val_acc / len(test_loader)

print(f'Test Loss: {avg_val_loss:.4f}, Test Acc: {avg_val_acc:.2f}')
torch.cuda.empty_cache()

Total parameters: 23528522
Trainable parameters: 23528522


100%|██████████| 438/438 [02:14<00:00,  3.25it/s]


Epoch 1/3, Loss: 0.1906, ACC: 95.4481


100%|██████████| 438/438 [01:55<00:00,  3.78it/s]


Epoch 2/3, Loss: 0.0107, ACC: 99.7860


100%|██████████| 438/438 [02:02<00:00,  3.56it/s]


Epoch 3/3, Loss: 0.0039, ACC: 99.9287


100%|██████████| 188/188 [00:36<00:00,  5.21it/s]


Test Loss: 0.0090, Test Acc: 99.73


In [10]:
# VGG16
weights = VGG16_Weights.DEFAULT
model = models.vgg16(weights=weights)
num_classes = 10  # Change this according to your dataset

# Modify final fully connected layer
num_features = model.classifier[6].in_features
model.classifier[6] = torch.nn.Linear(num_features, num_classes)
# Define loss function
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.to(device)
# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 3  # Change this as needed
model = model.to(device)

# check model size
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    acc = 0.0

    for images, labels in tqdm(train_loader):
        # Move inputs and labels to GPU if available
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        try:
            outputs = model(images)
            loss = criterion(outputs, labels)
    
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            acc += calculate_accuracy(outputs, labels)

        except RuntimeError as exception:
          if "out of memory" in str(exception):
              print('WARNING: out of memory, will pass this')     
              torch.cuda.empty_cache()
              continue
          else:
              raise exception
    # Calculate average loss and accuracy over an epoch
    avg_loss = running_loss / len(train_loader)
    avg_acc = acc / len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, ACC: {avg_acc:.4f}")

# test
model.eval()
val_running_loss = 0.0
val_acc = 0.0

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        
        val_running_loss += loss.item()
        val_acc += calculate_accuracy(outputs, labels)

avg_val_loss = val_running_loss / len(test_loader)
avg_val_acc = val_acc / len(test_loader)

print(f'Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.2f}')
torch.cuda.empty_cache()

Total parameters: 134301514
Trainable parameters: 134301514


100%|██████████| 438/438 [05:58<00:00,  1.22it/s]


Epoch 1/3, Loss: 0.2137, ACC: 92.4229


100%|██████████| 438/438 [06:24<00:00,  1.14it/s]


Epoch 2/3, Loss: 0.0621, ACC: 98.4446


100%|██████████| 438/438 [06:32<00:00,  1.12it/s]


Epoch 3/3, Loss: 0.0333, ACC: 99.1581


100%|██████████| 188/188 [00:34<00:00,  5.47it/s]

Val Loss: 0.0129, Val Acc: 99.67
